In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
from functools import partial
from dask.local import get_async
from dask import bag as db
from dask import array as da
import sprite2
import numpy as np
from matplotlib import pyplot as plt
import sprite2.aws
import sprite2.dask
import os

In [3]:
# set up logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)-5s %(asctime)s %(threadName)s: %(message)s')
logging.debug("test1")

root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)

logger = logging.getLogger('sprite2')
logger.setLevel(logging.DEBUG)

DEBUG 2018-04-26 09:17:44,085 MainThread: test1


### call a function remotely

In [4]:
def get_lambda_name():
    return os.environ.get('AWS_LAMBDA_FUNCTION_NAME', 'dummy')

In [5]:
# invoke locally, local environment doesnt define this var
%time get_lambda_name()

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 16 µs


'dummy'

In [6]:
# this func will be executed remotely
get_remote_lambda_name = sprite2.aws.remote_invoke(get_lambda_name, request_id=10101)

In [7]:
%time get_remote_lambda_name()

DEBUG 2018-04-26 09:17:44,587 MainThread: 10101 sending 977.0bytes
DEBUG 2018-04-26 09:17:44,666 MainThread: 10101 received 44.0bytes


CPU times: user 39.2 ms, sys: 8.35 ms, total: 47.6 ms
Wall time: 524 ms


'sprite'

### call a lambda remotely

In [8]:
# invoke remotely, lamba execution enfironment does
sprite2.aws.remote_invoke(lambda: 'im executing remotely')()

DEBUG 2018-04-26 09:17:44,687 MainThread: sending 860.0bytes
DEBUG 2018-04-26 09:17:44,709 MainThread: received 64.0bytes


'im executing remotely'

### remote errors are raised locally

In [9]:
def fun(): 
    raise Exception('awha?')
    
sprite2.aws.remote_invoke(fun)()

DEBUG 2018-04-26 09:17:44,728 MainThread: sending 872.0bytes
DEBUG 2018-04-26 09:17:44,750 MainThread: received 259.0bytes


AwsError: invalid response: {'errorMessage': 'awha?', 'errorType': 'Exception', 'stackTrace': [['/var/task/sprite2/aws.py', 99, 'executor', 'result = function()'], ['/Users/sayre/dev/sandbox/sprite2/sprite2/aws.py', 80, '<lambda>', ''], ['<ipython-input-9-c0fe0a063c87>', 2, 'fun', '']]}

### I can even numpy remotely

In [10]:
remote_square = sprite2.aws.remote_invoke(np.square)

In [11]:
%time remote_square(np.arange(10))

DEBUG 2018-04-26 09:17:57,352 MainThread: sending 1022.0bytes
DEBUG 2018-04-26 09:17:57,376 MainThread: received 332.0bytes


CPU times: user 9.61 ms, sys: 1.45 ms, total: 11.1 ms
Wall time: 31 ms


array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])